In [15]:
import faiss
import numpy as np
import time
import networkx as nx

try:
    res = faiss.StandardGpuResources()
except AttributeError:
    res = None

In [13]:
def load_state(path='./data/states/12340689.txt'):
    with open(path, 'r') as f:
        d = f.read().split("\n")[:-1]
        
    positions, forces, ticker = [], [], 0
    pos, forc = [], []
    for item in d:
        if ticker < 3:
            pos.append(float(item))
        else:
            forc.append(float(item))
        ticker += 1
        if ticker > 5:
            ticker = 0
            positions.append(pos), forces.append(forc)
            pos, forc = [], []
            
    return np.array(positions), np.array(forces)

def build_nn_graph(p, k=6, g=None, use_gpu=True):
    if g is None:
        g = nx.Graph()
#     t1 = time.time()
    p = p.astype("float32")
    index_flat = faiss.IndexFlatL2(3)
    if use_gpu:
        index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)
    index_flat.add(p)
    _, I = index_flat.search(p, k+1)
    I = I[:, 1:]
#     print(time.time() - t1)
#     edges = set()
    for i in range(len(p)):
        row = I[i, :]
        for item in row:
            edge = (i, item)
            edge = (min(edge), max(edge))
            if edge[0] != edge[1]:
                g.add_edge(edge[0], edge[1])
    return g

def build_random_graph(p, n_edges_per_node=3, g=None):
    if g is None:
        g = nx.Graph()
    _p = n_edges_per_node/len(p)
    _g = nx.erdos_renyi_graph(len(p), _p)
    g = nx.compose(g, _g)
    return g

def build_throughline_graph(p, g=None):
    if g is None:
        g = nx.Graph()
    last_node = len(p)
    for i in range(last_node):
        other = i+1
        if other == last_node:
            print('reached last node...')
            other = 0
        g.add_edge(i, other)
    return g

def build_graph(p, n_nn_edges=6, n_random_edges=3, do_build_throughline_graph=True, use_gpu=False):
    g = nx.Graph()
    if n_nn_edges > 0:
        g = build_nn_graph(p, k=n_nn_edges, g=g, use_gpu=use_gpu)
    if n_random_edges > 0:
        g = build_random_graph(p, n_edges_per_node=n_random_edges, g=g)
    if do_build_throughline_graph:
        g = build_throughline_graph(p, g)
    return g
    
    
p, f = load_state()
# I = build_nn_graph(p, use_gpu=False)
g = build_graph(p)
print(g.edges())

reached last node...
[(0, 24), (0, 6), (0, 22), (0, 1), (0, 27), (0, 58), (0, 30), (0, 114), (0, 127), (24, 1), (24, 4), (24, 6), (24, 58), (24, 34), (24, 30), (24, 59), (24, 85), (24, 108), (24, 23), (24, 25), (6, 4), (6, 30), (6, 9), (6, 1), (6, 5), (6, 12), (6, 7), (22, 20), (22, 25), (22, 73), (22, 10), (22, 37), (22, 39), (22, 53), (22, 61), (22, 15), (22, 28), (22, 40), (22, 69), (22, 21), (22, 23), (1, 4), (1, 36), (1, 10), (1, 34), (1, 96), (1, 113), (1, 2), (27, 7), (27, 13), (27, 19), (27, 30), (27, 63), (27, 55), (27, 58), (27, 73), (27, 92), (27, 26), (27, 28), (58, 7), (58, 13), (58, 34), (58, 63), (58, 94), (58, 99), (58, 106), (58, 3), (58, 48), (58, 57), (58, 59), (4, 36), (4, 10), (4, 42), (4, 28), (4, 34), (4, 25), (4, 97), (4, 109), (4, 113), (4, 3), (4, 5), (36, 28), (36, 42), (36, 60), (36, 10), (36, 96), (36, 48), (36, 72), (36, 41), (36, 104), (36, 35), (36, 37), (10, 25), (10, 64), (10, 20), (10, 61), (10, 32), (10, 96), (10, 9), (10, 11), (34, 96), (34, 60), (3

In [44]:
p = np.random.random((10000, 3)).astype('float32')

t1 = time.time()
index_flat = faiss.IndexFlatL2(3) 
gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)
# index = faiss.IndexIVFFlat(3)
gpu_index_flat.add(p)
_, I = gpu_index_flat.search(p, 7)
# print(I[:, 1:])
print(time.time() - t1)

0.006889820098876953


In [2]:
import numpy as np
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.


64

In [20]:
import numpy as np
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

import faiss                   # make faiss available
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)
k = 4                          # we want to see 4 nearest neighbors
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)
D, I = index.search(xq, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])  

True
100000
[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]
[[0.        7.175174  7.2076287 7.251163 ]
 [0.        6.323565  6.684582  6.799944 ]
 [0.        5.7964087 6.3917365 7.2815127]
 [0.        7.277905  7.5279875 7.6628447]
 [0.        6.763804  7.295122  7.368814 ]]
[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]


In [17]:
g.edges()

EdgeView([(0, 24), (0, 6), (0, 22), (0, 1), (0, 27), (0, 58), (0, 30), (0, 114), (0, 127), (24, 1), (24, 4), (24, 6), (24, 58), (24, 34), (24, 30), (24, 59), (24, 85), (24, 108), (24, 23), (24, 25), (6, 4), (6, 30), (6, 9), (6, 1), (6, 5), (6, 12), (6, 7), (22, 20), (22, 25), (22, 73), (22, 10), (22, 37), (22, 39), (22, 53), (22, 61), (22, 15), (22, 28), (22, 40), (22, 69), (22, 21), (22, 23), (1, 4), (1, 36), (1, 10), (1, 34), (1, 96), (1, 113), (1, 2), (27, 7), (27, 13), (27, 19), (27, 30), (27, 63), (27, 55), (27, 58), (27, 73), (27, 92), (27, 26), (27, 28), (58, 7), (58, 13), (58, 34), (58, 63), (58, 94), (58, 99), (58, 106), (58, 3), (58, 48), (58, 57), (58, 59), (4, 36), (4, 10), (4, 42), (4, 28), (4, 34), (4, 25), (4, 97), (4, 109), (4, 113), (4, 3), (4, 5), (36, 28), (36, 42), (36, 60), (36, 10), (36, 96), (36, 48), (36, 72), (36, 41), (36, 104), (36, 35), (36, 37), (10, 25), (10, 64), (10, 20), (10, 61), (10, 32), (10, 96), (10, 9), (10, 11), (34, 96), (34, 60), (34, 82), (34,